In [ ]:
# default_exp data_scrambler

# data_scrambler

> Data scrambler.

In [ ]:
# export
class DataScrambler:
    def __init__(self):
        pass

In [ ]:
# export
class configuration:
    def __init__(self, att_name: str, to_scramble: float, mistake_epsilon:float, uniform: bool):
        self.attName = att_name
        self.toScramble = to_scramble
        self.mistakeEpsilon = mistake_epsilon
        self.uniform = uniform
